<h1> Problem 75 - Singular Integer Right Triangles </h1> 

It turns out that 12 cm is the smallest length of wire that can be bent to form an integer sided right angle triangle in exactly one way, but there are many more examples.
\begin{align*}
    (3,4,5) & = 12 \\
    (6,8,10) & = 24 \\
    (5,12,13) & = 30 \\
    (9,12,15) & = 36 \\
    (8,15,17) & = 40 \\
    (12,16,20) & = 48 
\end{align*}
In contrast, some lengths of wire, like 20 cm, can't be bent to form an integer sided right angle triangle, and other lenths allow more than one solution to be found; for example, using 120 cm it is possible to form exactly three different integer sided right angle triangles. 
\begin{equation*}
    (30,40,50) = (20,48,52) = (24,45,51) = 120
\end{equation*}
Given that $L$ is the length of the wire, for how many values of $L \leq 1500000$ can exactly one integer sided right angle triangle be formed. 

---


Properties to exploit:
1. A property of the sum of pythagorean triplets is that it will always be even. This automatically eliminates half of all possible values. 
2. From one primitive pythagorean triplet, I can find all sums that are a multiple of it. If a number is a multiple of more than one sum, then it will have more than one solution. For example, $120 = 30 \times 4 = 12 \times 10 = 40 \times 3$. Therefore, from the first $x$ number of primitives, I can eliminate most sums.
3. I can terminate when every even number less than $1500000$ can be expressed as a multiple of my primitives. I don't think I need to go all the way to $1500000$. 

The real question is, is this equivalent to just finding all the primitives? I think if I solve for all the pythogorean triplets up to, 
\begin{equation*}
    w = 2m(m+n)
\end{equation*}
where $w$ is the perimeter, it should be fine. 

[Euclid's formula to generate Pythagorean Triplets](https://en.wikipedia.org/wiki/Pythagorean_triple#Generating_a_triple), states that, for an aribitrary pair of integers, $m$ and $n$, with $m>n>0$, then 
\begin{equation*}
    x = m^2 - n^2 \qquad y = 2mn \qquad z = m^2 + n^2
\end{equation*}
where $(x, y, z)$ is a pythagorean triplet that fulfills $x^2 + y^2 = z^2$, $m$ and $n$ are coprime (i.e. $gcd(m,n) = 1$) integers, and $m$ and $n$ are of opposite parity (i.e. if $m$ is even, then $n$ is odd, and vice versa). 

[All pairs of coprime intergers](https://en.wikipedia.org/wiki/Coprime_integers#Generating_all_coprime_pairs), $m$ and $n$ with $m>n$ are known to be given by, 
\begin{align}
    (2m - n ,m) & \\
    (2m + n ,m) & \\
    (m + 2n, n)
\end{align}
where each equation represents one branch of a [ternary tree](https://en.wikipedia.org/wiki/Ternary_tree). This scheme is known to be exhaustive and non-redundant with not invalid memebers [cite](https://en.wikipedia.org/wiki/Coprime_integers#Generating_all_coprime_pairs). Since pythagorean triplets require either odd-even pairs or even-odd pairs, then the starting point for each branch will be $(2,1)$.

It can be guaranteed that the $m$ and $n$ values will be larger than the previous iteration, one can safely assume that $x$, $y$ and $z$ will be larger and hence their sum will be larger. 

Actually, if I just need to find the number of primitives, then I don't ever need to find the triplets. I can just find the number of terms that fulfill the coprime conditions. 


In [2]:
use std::collections::HashMap;
use std::error::Error;

#[derive(Hash, Eq, PartialEq, Debug, Default, Clone)]
struct PythagoreanTriplet {
    branch_1: (u32, u32),
    branch_2: (u32, u32),
    branch_3: (u32, u32),
    perimeter: Vec<u32>,
    which_branches_alive: Vec<bool>,
    are_all_branches_dead: bool,
    number_of_alive_branches: u8,
}

impl PythagoreanTriplet {
    fn new(m_0: &u32, n_0: &u32, branch_status: &Vec<bool>, max_value: u32) -> PythagoreanTriplet{
        let mut output = PythagoreanTriplet::default();
        
        output.which_branches_alive = branch_status.clone();
        output.perimeter = vec![0;3];
        
        for i in 0..3 {
            if branch_status[i] {
                output.compute_for_branch(i + 1, *m_0, *n_0).unwrap();
            } else {
                output.set_dead_branch_values(i + 1).unwrap();
            }
        }
        
        let buffer = (0.1 * max_value as f64) as u32;
        output.which_branches_alive = output.perimeter
                                                .iter()
                                                .zip(branch_status)
                                                .map(|(p, initial_status)| 
                                                    if p > &(max_value + buffer) || initial_status == &false 
                                                        {false} 
                                                    else {true})
                                                .collect();
        output.number_of_alive_branches = output.which_branches_alive.iter().filter(|&x| x == &true).count() as u8;
        println!("{:?} {:?}", output.which_branches_alive, output.number_of_alive_branches);
        
        if output.number_of_alive_branches == 0 {
            output.are_all_branches_dead = true;
        } else {
            output.are_all_branches_dead = false;
        }
        output
    }
    
    fn get_number_alive_branches(&self) -> u8 {
        self.number_of_alive_branches
    }
    
    fn get_are_all_branches_dead(&self) -> bool {
        self.are_all_branches_dead
    }
    
    fn get_branch_status(&self) -> &Vec<bool> {
        &self.which_branches_alive
    }
    
    fn set_dead_branch_values(& mut self, branch_num: usize) 
        -> Result<& mut PythagoreanTriplet, Box<dyn Error>> 
    {
        let error_message = "Branch number must be 1, 2, or 3";
            
        match branch_num {
            1 => self.branch_1 = (0,0),
            2 => self.branch_2 = (0,0),
            3 => self.branch_3 = (0,0),
            _ => return Err(Box::<dyn Error>::from(error_message)),
        };
            
        return Ok(self)
    }
    
    fn compute_for_branch(&mut self, branch_num: usize, m: u32, n: u32)
        -> Result<& mut PythagoreanTriplet, Box<dyn Error>> 
    {
        let error_message = "Branch number must be 1, 2, or 3";
        match branch_num {
            1 => {
                let new_val = PythagoreanTriplet::next_branch_1_pair(m, n);
                self.perimeter[branch_num - 1] = PythagoreanTriplet::compute_perimeter(new_val.0, new_val.1);
                self.branch_1 = new_val;
            },
            2 => {
                let new_val = PythagoreanTriplet::next_branch_2_pair(m, n);
                self.perimeter[branch_num - 1] = PythagoreanTriplet::compute_perimeter(new_val.0, new_val.1);
                self.branch_2 = new_val;
            },
            3 => {
                let new_val = PythagoreanTriplet::next_branch_3_pair(m, n);
                self.perimeter[branch_num - 1] = PythagoreanTriplet::compute_perimeter(new_val.0, new_val.1);
                self.branch_3 = new_val;
            },
            _ => return Err(Box::<dyn Error>::from(error_message)),
        };
        Ok(self)
    }
    
    fn compute_perimeter(m: u32, n: u32) -> u32 {
        2*m*(m+n)
    }
    
    fn is_max_perimeter_exceeded(max_perimeter: u32, m_and_n: (u32, u32)) -> bool {
        let perimeter = PythagoreanTriplet::compute_perimeter(m_and_n.0, m_and_n.1);
//         println!("m = {} n = {} perimeter = {}", m_and_n.0, m_and_n.1, perimeter);
        if perimeter > max_perimeter {
            true
        } else {
            false
        } 
    }
    
//     fn test_if_all_future_branches_will_exceed_maximum_perimeter(m: u32, n:u32, max_peri: u32) -> bool {
//         let num_exceed = (1..4).into_iter()
//                               .map(|x| match x {
//                                   1 => PythagoreanTriplet::next_branch_1_pair(m, n),
//                                   2 => PythagoreanTriplet::next_branch_2_pair(m, n),
//                                   3 => PythagoreanTriplet::next_branch_3_pair(m, n),
//                                 })
//                               .filter(|(new_m, new_n)| PythagoreanTriplet::compute_perimeter(*new_m, *new_n) > max_peri)
//                               .count();
//         if num_exceed == 3 {
//             return true
//         }
//         false
        
//     }
    
    fn next_branch_1_pair(m: u32, n: u32) -> (u32, u32) {
        (2*m -n , m)
    }

    fn next_branch_2_pair(m: u32, n:u32) -> (u32, u32) {
        (2*m + n, m)
    }

    fn next_branch_3_pair(m: u32, n:u32) -> (u32, u32) {
        (m + 2*n, n)
    }
    
}

In [3]:
struct PythagoreanTripletIterator<'a> {
    triplet: &'a PythagoreanTriplet, 
    index: usize,
}

struct BranchInfo {
    m: u32,
    n: u32, 
    alive: bool,
}

impl BranchInfo {
    fn new(m: u32, n: u32, alive:bool) -> BranchInfo {
        BranchInfo{m, n, alive}
    }
}


impl<'a> Iterator for PythagoreanTripletIterator<'a> {
    type Item = BranchInfo;
    
    fn next(&mut self) -> Option<BranchInfo> {
        let branch_value = match self.index {
            0 => self.triplet.branch_1,
            1 => self.triplet.branch_2,
            2 => self.triplet.branch_3,
            _ => return None, 
        };
        let branch_status = self.triplet.get_branch_status()[self.index];
        self.index += 1;
        
        Some(BranchInfo::new(branch_value.0, branch_value.1, branch_status))
    }
}

impl PythagoreanTriplet {
    fn iter(&self) -> PythagoreanTripletIterator {
        PythagoreanTripletIterator{
            triplet: self,
            index: 0
        }
    }
}


In [4]:
fn create_key_name(generation_number: u32, child_number: u32) -> String {
    String::from(vec![generation_number.to_string(), child_number.to_string()].join("_"))
}

In [5]:
use std::collections::HashMap;

fn assemble_primitives(max_value: u32) -> (HashMap<String, PythagoreanTriplet>,u32) {
// fn assemble_primitives(max_value: u32) -> u32 {
    let mut all_triplets = HashMap::new();
    
    let mut number_of_primitives: u32 = 1;
    
    let mut which_generation = 0;
    let mut which_child_in_generation = 0;
    
    let initial_m = 2;
    let initial_n = 1;
    let initial_status = vec![true;3];
    
    let mut key_name = create_key_name(which_generation, which_child_in_generation);
    
    all_triplets.insert(key_name, PythagoreanTriplet::new(&initial_m, &initial_n, &initial_status, max_value));
    
    loop {
        let key_subset =  format!("{}_", which_generation);
        
        let children_in_this_generation: HashMap< String,  PythagoreanTriplet> = all_triplets
                                                                                    .clone()
                                                                                    .into_iter()
                                                                                    .filter(|(k, _)| k.contains(& key_subset))
                                                                                    .collect::<HashMap<String, PythagoreanTriplet>>();
//         println!("{:?}", &children_in_this_generation);
        
        let number_of_live_children = children_in_this_generation.len();
        
//         println!("{:?}", number_of_live_children);
        
        if number_of_live_children == 0 {
            return (all_triplets, number_of_primitives)
        }
        
        // Create the next generation 
        which_generation += 1;
        which_child_in_generation = 0;
        
        for (_, parent_triplet) in children_in_this_generation.iter().filter(|(_, v)| !v.get_are_all_branches_dead()){
            
            for branch in parent_triplet.iter().filter(|x| x.alive) {
                
                key_name = create_key_name(which_generation, which_child_in_generation);
                all_triplets.insert(key_name, PythagoreanTriplet::new(&branch.m, &branch.n, parent_triplet.get_branch_status(), max_value));
                which_child_in_generation += 1;
                number_of_primitives += 1;
                
            }
                
        }
    }
}


In [6]:
fn extract_perimeters(primitives: &HashMap<String, PythagoreanTriplet>) -> Vec<u32> {
    primitives.iter().flat_map(|(_, v)| v.iter()
                                    .filter(|branch| branch.alive)
                                    .map(|branch| 2*branch.m*(branch.m + branch.n))
                                    .collect::<Vec<u32>>()
                                    )
                     .collect::<Vec<u32>>()
    
}

In [7]:
// let n = 1500000;
let n = 100;
let primitives = assemble_primitives(n);
let all_perimeters = extract_perimeters(&primitives.0);
println!("{} {:?} {:?}", n, primitives, all_perimeters);
for (key, item) in primitives.0.iter() {
    println!("{} {} {:?} {:?} {:?}", key, item.number_of_alive_branches, item.branch_1, item.branch_2, item.branch_3)
};

[true, true, true] 3
[true, false, false] 1
[false, false, false] 0
[false, false, true] 1
[true, false, false] 1
[false, false, false] 0
[false, false, false] 0
100 ({"1_0": PythagoreanTriplet { branch_1: (4, 3), branch_2: (8, 3), branch_3: (7, 2), perimeter: [56, 176, 126], which_branches_alive: [true, false, false], are_all_branches_dead: false, number_of_alive_branches: 1 }, "0_0": PythagoreanTriplet { branch_1: (3, 2), branch_2: (5, 2), branch_3: (4, 1), perimeter: [30, 70, 40], which_branches_alive: [true, true, true], are_all_branches_dead: false, number_of_alive_branches: 3 }, "1_1": PythagoreanTriplet { branch_1: (8, 5), branch_2: (12, 5), branch_3: (9, 2), perimeter: [208, 408, 198], which_branches_alive: [false, false, false], are_all_branches_dead: true, number_of_alive_branches: 0 }, "2_1": PythagoreanTriplet { branch_1: (0, 0), branch_2: (0, 0), branch_3: (8, 1), perimeter: [0, 0, 144], which_branches_alive: [false, false, false], are_all_branches_dead: true, number_of_al